In [1]:
import geopandas as gpd
from rt_stop_times import *
from shared_utils import gtfs_utils_v2

In [2]:
rt_schedule_stop_times = assemble_scheduled_rt_stop_times(
    "2025-04-16",
    [*GTFS_DATA_DICT.rt_stop_times.trip_stop_cols]
)

### Get an example trip with non-monotonic stop_sequence values

In [16]:
example_trip = rt_schedule_stop_times.loc[
    (rt_schedule_stop_times.schedule_gtfs_dataset_key == "c65bd95ac0009a74df9ff840fc416771")
    & (rt_schedule_stop_times.trip_id == "902110")
].sort_values("stop_sequence")
example_trip["rt_non_monotonic"] = (
    example_trip["rt_arrival_sec"].shift(1) > example_trip["rt_arrival_sec"]
)
example_trip[["stop_sequence", "scheduled_arrival_sec", "rt_arrival_sec", "rt_non_monotonic"]]

,stop_sequence,scheduled_arrival_sec,rt_arrival_sec,rt_non_monotonic
463891,2,37800.0,37707,False
463880,3,37832.0,37691,True
463854,5,37931.0,37818,False
463879,6,37965.0,37912,False
463855,7,38010.0,37963,False
463859,8,38082.0,38031,False
463864,9,38116.0,38065,False
463863,10,38182.0,38122,False
463888,11,38237.0,38171,False
463867,12,38309.0,38221,False


### Get a list of agencies that have trips with rt times and not scheduled times

In [4]:
agencies_with_nonscheduled_service = rt_schedule_stop_times.loc[
    
    (rt_schedule_stop_times.scheduled_arrival_sec.isna())
    & ~(rt_schedule_stop_times.rt_arrival_sec.isna())
].schedule_gtfs_dataset_key.unique()

In [5]:
agencies_with_nonscheduled_service

array(['2f1c266fc20f9875777fb752af32a66e',
       '0a3c0b21c85fb09f8db91599e14dd7f7',
       'ac2951bfaa7ecf6b80ba9e50aef1ae86',
       '0f5e1b251db53223200c5bfc365d33f2',
       'a8d5f90bfd689badb7e1deb041408e96',
       '78b44303c1714f6c6a4801637c2a5c9d',
       '4be5df8915abb52a9e86a7168403f6d6',
       'd2b09fbd392b28d767c28ea26529b0cd',
       '53c2df3f17447b687a57aaf91918bead',
       'e8d0fd2f1c4b13707a24909a0f206271',
       'cb8a465cffec67c8fd90f31b389ed4c3',
       'a23f73c5f192be7fdc1a7dea4595038d',
       'fc6cd27871cce0092a08ccf68fb240a2',
       '4e2936d8f27a9bca79289ec062a1691a',
       'ea65e81b31025ca3e74e8ffb27e1a223',
       'a253a8d7acd57657bb98050f37dd6b0f',
       '205d13dc0fa95f904ea9bedd384509c7',
       'b9f9ee9267bd3564d5d2cfbe2389f3fa',
       '79c9d44937498d0aa50d58f3868a941a',
       '5ed4b903a3c6049509b935883c440209',
       'acf268b2ba5b0dedba66383083cb22b7'], dtype=object)

In [6]:
gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(
        selected_date="2025-04-16", keep_cols=["name", "gtfs_dataset_key"]
).set_index("gtfs_dataset_key").loc[agencies_with_nonscheduled_service]

,name
gtfs_dataset_key,
2f1c266fc20f9875777fb752af32a66e,LAX Flyaway Bus Schedule
0a3c0b21c85fb09f8db91599e14dd7f7,Lake Schedule
ac2951bfaa7ecf6b80ba9e50aef1ae86,Nevada County Schedule
0f5e1b251db53223200c5bfc365d33f2,Bay Area 511 Fairfield and Suisun Transit Sche...
a8d5f90bfd689badb7e1deb041408e96,Bear Schedule
78b44303c1714f6c6a4801637c2a5c9d,Bay Area 511 WestCAT Schedule
4be5df8915abb52a9e86a7168403f6d6,Tehama Schedule
d2b09fbd392b28d767c28ea26529b0cd,Unitrans Schedule
53c2df3f17447b687a57aaf91918bead,MV Shuttle Schedule
